In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#append repo folder to search path
sys.path.append('/content/drive/MyDrive/limb-position-EMG-Repo/')
from utils import *



In [8]:
def log_reg_xsubject_join_data(feature_matrix, target_labels, sub_labels, block_labels, model_dict, n_splits = 4,\
                       verbose = 0, epochs = 40, batch_size = 2, validation_split = 0.1, mv = False, permute = False):
    #subjects in list. there are the units over which we will do train/test split
    subs = np.unique(sub_labels)

    #initialize object for k-fold cross-validation
    kf = KFold(n_splits=n_splits,shuffle = True)
    #initialize empty array
    train_f1_scores = np.empty((n_splits,))
    test_f1_scores = np.empty((n_splits,))
    train_history = dict()
    train_history['loss'] = np.empty((0,0))
    train_history['val_loss'] = np.empty((0,0))

    for split_count, (subs_train_idxs, subs_test_idxs) in enumerate(kf.split(subs)):
        print('Split Count: %i'% (split_count+1))

        #get train and test indices
        train_subs = subs[subs_train_idxs]
        test_subs = subs[subs_test_idxs]
        train_idxs = np.where(np.isin(sub_labels,train_subs, invert = False))[0]
        test_idxs = np.where(np.isin(sub_labels,test_subs, invert = False))[0]

        #get trained model
        train_f1, trained_model, scaler, history = get_trained_model(feature_matrix, target_labels, train_idxs,model_dict, model_dict,\
                                                                    verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                    validation_split = validation_split,\
                                                                    mv = mv, permute = permute)
        #Evaluating on held-out subjects
        test_f1 = evaluate_trained_log_reg(feature_matrix, target_labels, test_idxs, exclude, trained_model, scaler, mv = mv)

        #put scores in array
        train_f1_scores[split_count] = train_f1
        test_f1_scores[split_count] = test_f1

        #append history
        train_history['loss'] = np.vstack((train_history['loss'],history.history['loss'])) if train_history['loss'].size else np.array(history.history['loss'])
        if validation_split>0:
            train_history['val_loss'] = np.vstack((train_history['val_loss'],history.history['val_loss'])) if train_history['val_loss'].size else np.array(history.history['val_loss'])

    return train_f1_scores, test_f1_scores, train_history

def get_trained_model(X, Y, train_idxs, exclude = [], model_dict = {}, verbose = 0, epochs = 40, batch_size = 2,\
                      validation_split = 0, mv = False, permute = False):

    if not model_dict:
        model_dict = {'n_dense_pre':0, 'activation':''}
    #get training data cubes
    X_train_cube, Y_train_cube, scaler = prepare_data_for_log_reg(X,Y, train_idxs, exclude, train = True)
    if permute:
        perm_idxs = np.random.permutation(np.arange(Y_train_cube.shape[0]))
        Y_train_cube = Y_train_cube[perm_idxs,:]

    #testfor equal number of samples
    assert X_train_cube.shape[0] == Y_train_cube.shape[0]
    n_features, n_outputs = X_train_cube.shape[1], Y_train_cube.shape[1]
    #setting timestep dimension to None 
    model = get_log_reg_model((n_features,),n_outputs, n_dense_pre=model_dict['n_dense_pre'], activation=model_dict['activation'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary

    print('Training Model')
    # fit network
    history = model.fit(X_train_cube, Y_train_cube,validation_split = validation_split, \
                        epochs=epochs, batch_size=batch_size, verbose=verbose)
    # # evaluate trained network
    print('Evaluate Model on Trained Data')

    if mv:
        # get testing data cubes
        X_test_cube, Y_test_cube, scaler = prepare_data_for_log_reg(X,Y, train_idxs, [], train = False, scaler = scaler)
        y_pred = get_mv_preds(X_test_cube, model, n_votes= 5)+1
        y_true = np.squeeze(np.argmax(Y_test_cube,1))
        include_idxs = np.where(np.isin(y_true,exclude, invert = True))[0]
        y_true = y_true[include_idxs]
        y_pred = y_pred[include_idxs]
        train_f1 = f1_score(y_true,y_pred,average = 'weighted')

    else:
        #get score for training data
        train_f1 = get_log_reg_f1(X_train_cube, Y_train_cube, model)
    return train_f1, model, scaler, history

def plot_training_history(train_history, fig_title,fig_fn):
    
    #make dataframe
    history_df = []
    n_folds, n_epochs = train_history['loss'].shape
    for f in range(n_folds):
        history_df.append(pd.DataFrame({'Loss': train_history['loss'][f,:],\
                        'Epoch':np.arange(n_epochs)+1,\
                        'Fold':[f+1 for x in range(n_epochs)],\
                        'Type':['Train' for x in range(n_epochs)]}))
        history_df.append(pd.DataFrame({'Loss': train_history['val_loss'][f,:],\
                        'Epoch':np.arange(n_epochs)+1,\
                        'Fold':[f+1 for x in range(n_epochs)],\
                        'Type':['Val' for x in range(n_epochs)]}))

    history_df = pd.concat(history_df,axis = 0)


    #make figure
    sns.relplot(data=history_df, x = 'Epoch', y = 'Loss', col = 'Fold',col_wrap = 2, style = 'Type',\
                style_order = ['Train','Val'],kind = 'line')
    fig = plt.gcf()
    fig.suptitle(fig_title, y = 1.05)
    fig.savefig(fig_fn,dpi = 300)
    plt.close()

In [4]:
#define hyper params for each model
model_dict = {1:{'n_dense_pre':0, 'activation':''},\
              2:{'n_dense_pre':1, 'activation':'tanh'},\
              3:{'n_dense_pre':1, 'activation':'relu'},\
              4:{'n_dense_pre':2, 'activation':'tanh'},\
              5:{'n_dense_pre':2, 'activation':'relu'},\
              }


In [5]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/'

nsubjects = 36

#randomly-selected subjects to use as hold-out test data 
test_subjects = [17, 23,  7,  8,  3]

# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap

#excluded labels
exclude = [0,7]

In [6]:
#intialize empty lists
feature_matrix_all = np.empty((0,0))
target_labels_all = np.empty((0,))
window_tstamps_all = np.empty((0,))
block_labels_all  = np.empty((0,))
subject_id_all = np.empty((0,))
block_count = 0

for subject_id in range(1,nsubjects+1):
    if subject_id not in test_subjects:
        subject_folder = os.path.join(data_folder,'%02d'%(subject_id))
        print('=======================')
        print(subject_folder)

        # Process data and get features 
        #get features across segments and corresponding info
        feature_matrix, target_labels, window_tstamps, \
        block_labels, series_labels = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                        win_size, step)

        #prevent repeat of block labels by increasing block count
        block_labels = block_labels+block_count
        block_count = np.max([block_count, np.max(block_labels)])
        #exclude indicated samples
        in_samples = np.where(np.isin(target_labels,exclude, invert = True))[0]
        feature_matrix_in = feature_matrix[in_samples,:]
        target_labels_in = target_labels[in_samples]
        window_tstamps_in = window_tstamps[in_samples]
        block_labels_in = block_labels[in_samples]
        # concatenate lists
        feature_matrix_all = np.vstack((feature_matrix_all,feature_matrix_in)) if feature_matrix_all.size else feature_matrix_in
        target_labels_all = np.hstack((target_labels_all,target_labels_in))
        window_tstamps_all = np.hstack((window_tstamps_all,window_tstamps_in))
        block_labels_all = np.hstack((block_labels_all,block_labels_in))
        subject_id_all = np.hstack((subject_id_all,np.ones((block_labels_in.size))*subject_id))
        

/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/01
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/02
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/04
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/05
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/06
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/09
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/10
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/11
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/12
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/13
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/14
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/15
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/16
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/18
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/19
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/20
/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/21
/content/drive

In [9]:
results_folder = '/content/drive/MyDrive/limb-position-EMG-Repo/results_data/xsubject_joint_data/log_reg/'
figure_folder = '/content/drive/MyDrive/limb-position-EMG-Repo/figures/training_history/xsubject_joint_data/log_reg'


#RNN training args - all other arguments are the same
verbose = 0
epochs = 30
batch_size = 5
validation_split = 0.1
# experiment params
n_splits = 4
nreps = 10
for model_id in range(2,5+1):
    np.random.seed(1)# Set seed for replicability
    results_df = []
    for rep in range(nreps):
        print('Model %d | Rep %d'%(model_id, rep+1))
        print('--True Data--')
        train_f1, test_f1, train_history = log_reg_xsubject_join_data(feature_matrix_all, target_labels_all, subject_id_all, block_labels_all,\
                                                                                model_dict = model_dict[model_id], n_splits = n_splits,\
                                                                                verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                validation_split = validation_split, mv = False, permute = False)
        #put results in datafram e
        results_df.append(pd.DataFrame({'F1_score':train_f1,\
                        'Fold':np.arange(train_f1.size)+1,\
                        'Rep':[rep+1 for x in range(train_f1.size)],\
                        'Type':['Train' for x in range(train_f1.size)],\
                        'Shuffled':[False for x in range(train_f1.size)]
                        }))

        results_df.append(pd.DataFrame({'F1_score':test_f1,\
                        'Fold':np.arange(test_f1.size)+1,\
                        'Rep':[rep+1 for x in range(test_f1.size)],\
                        'Type':['Test' for x in range(test_f1.size)],\
                        'Shuffled':[False for x in range(test_f1.size)]
                        }))
        #plot training history
        fig_title = 'Log reg model %02d; rep %i'%(model_id,rep)
        fig_fn = os.path.join(figure_folder,'log_reg_model_%02d_rep_%i_loss.png'%(model_id,rep))
        plot_training_history(train_history, fig_title,fig_fn)

        #repeat with shuffled data
        print('Model %d | Rep %d'%(model_id, rep+1))
        print('--Permuted Data--')
        train_f1_perm, test_f1_perm, train_history = log_reg_xsubject_join_data(feature_matrix_all, target_labels_all, subject_id_all, block_labels_all,\
                                                                            model_dict = model_dict[1], n_splits = n_splits,\
                                                                            verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                            validation_split = validation_split, mv = False, permute = True)
        
        results_df.append(pd.DataFrame({'F1_score':train_f1_perm,\
                        'Fold':np.arange(train_f1_perm.size)+1,\
                        'Rep':[rep+1 for x in range(train_f1_perm.size)],\
                        'Type':['Train' for x in range(train_f1_perm.size)],\
                        'Shuffled':[True for x in range(train_f1_perm.size)]
                        }))

        results_df.append(pd.DataFrame({'F1_score':test_f1_perm,\
                        'Fold':np.arange(test_f1_perm.size)+1,\
                        'Rep':[rep+1 for x in range(test_f1_perm.size)],\
                        'Type':['Test' for x in range(test_f1_perm.size)],\
                        'Shuffled':[True for x in range(test_f1_perm.size)]
                        }))
    #concatenate all data frames
    results_df = pd.concat(results_df,axis = 0)

    #save results to file
    results_fn = 'model_%02d_results.h5'%(model_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')


Model 2 | Rep 1
--True Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate Model
Model 2 | Rep 1
--Permuted Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate Model
Model 2 | Rep 2
--True Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate 

In [9]:
model_dict[2]['activation']

'tanh'